In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import pandas_datareader as pdr
from pandas_datareader import data, wb
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

In [2]:
df=pd.read_csv("NVDA.csv", decimal = ',')
df.head()

,Date,Open,Close,High,Low,Volume
0,4-Jan-16,32.290001,32.369999,32.580002,32.040001,8951900
1,5-Jan-16,32.98,32.889999,33.439999,32.5,12256800
2,6-Jan-16,32.349998,31.530001,32.5,31.16,11233600
3,7-Jan-16,30.74,30.280001,30.950001,29.879999,16132600
4,8-Jan-16,30.67,29.629999,30.700001,29.57,9961800


In [3]:
# Modelling CAPM and calculating alpha and beta from historical values.
'''
Download monthly prices of NVIDIA and S&P 500 index from 2016 to 2018
CSV file downloaded from Yahoo File
start period: 02/11/2016 
end period: 30/11/2018
period format: DD/MM/YEAR
'''

df_nvda = pdr.get_data_yahoo('NVDA', '2016-01-04', '2018-08-30')

#Collect data on the market from S&P 500 StockPrice data --- GSPC
df_gspc = pdr.get_data_yahoo('^GSPC', '2016-01-04', '2018-08-30')

In [4]:
df_nvda.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-04,32.580002,32.040001,32.290001,32.369999,8951900.0,31.724598
2016-01-05,33.439999,32.500000,32.980000,32.889999,12256800.0,32.234219
2016-01-06,32.500000,31.160000,32.349998,31.530001,11233600.0,30.901340
2016-01-07,30.950001,29.879999,30.740000,30.280001,16132600.0,29.676266
2016-01-08,30.700001,29.570000,30.670000,29.629999,9961800.0,29.039225


In [5]:
df_gspc.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-04,2038.199951,1989.680054,2038.199951,2012.660034,4304880000,2012.660034
2016-01-05,2021.939941,2004.170044,2013.780029,2016.709961,3706620000,2016.709961
2016-01-06,2011.709961,1979.050049,2011.709961,1990.260010,4336660000,1990.260010
2016-01-07,1985.319946,1938.829956,1985.319946,1943.089966,5076590000,1943.089966
2016-01-08,1960.400024,1918.459961,1945.969971,1922.030029,4664940000,1922.030029


In [6]:
# joining the closing prices of the two datasets 
monthly_prices = pd.concat([df_nvda['Open'], df_gspc['Open']], axis=1)
monthly_prices.columns = ['NVDA', '^GSPC']

In [7]:
monthly_prices.head()

,NVDA,^GSPC
Date,,
2016-01-04,32.290001,2038.199951
2016-01-05,32.980000,2013.780029
2016-01-06,32.349998,2011.709961
2016-01-07,30.740000,1985.319946
2016-01-08,30.670000,1945.969971


In [8]:
# calculate monthly returns
# Percentage change between the current and a prior element.
#Computes the percentage change from the immediately previous row by default. 
#This is useful in comparing the percentage of change in a time series of elements.

monthly_returns = monthly_prices.pct_change(1)  # 1 used to denote Periods to shift for forming percent change.
clean_monthly_returns = monthly_returns.dropna(axis=0)  # drop first missing row
clean_monthly_returns.head()

,NVDA,^GSPC
Date,,
2016-01-05,0.021369,-0.011981
2016-01-06,-0.019103,-0.001028
2016-01-07,-0.049768,-0.013118
2016-01-08,-0.002277,-0.019820
2016-01-11,-0.032931,-0.010201


In [9]:
#Ordinary Least Square (OLS)Regression with Statsmodels.
# split dependent and independent variable
X = clean_monthly_returns['^GSPC']
y = clean_monthly_returns['NVDA']


In [10]:
# Add a constant to the independent value
X1 = sm.add_constant(X)

# make regression model 
model = sm.OLS(y, X1)

# fit model and print results
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   NVDA   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     190.9
Date:                Sun, 15 Dec 2019   Prob (F-statistic):           2.24e-38
Time:                        19:05:22   Log-Likelihood:                 1585.5
No. Observations:                 671   AIC:                            -3167.
Df Residuals:                     669   BIC:                            -3158.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.001      2.911      0.0

C:\Users\gadep\OneDrive\Anakonda\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


The coefficient value for (^GSPC) is 1.6134. If the Beta value provided by Yahoo! Finance is anywhere as close to this figure, then our regression model and attempt to replicate how Yahoo! Finance calculates Beta values is correct.

In [11]:
# alternatively scipy linear regression

#https://finance.yahoo.com/quote/NVDA/

from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)

print(slope)

1.768894307438861


In [12]:
##model 2

def capm(start_date, end_date, ticker1, ticker2):
    df = pdr.get_data_yahoo(ticker1, start_date, end_date)
    dfb = pdr.get_data_yahoo(ticker2, start_date, end_date)
    df.head()
    dfb.head()
# create a time-series
    rts = df.resample('M').last()
    rbts = dfb.resample('M').last()
    dfsm = pd.DataFrame({'s_open' : rts['Open'], 'b_open' : rbts['Open']}, index=rts.index)

# compute returns
    dfsm[['s_returns', 'b_returns']] = dfsm[['s_open','b_open']]/dfsm[['s_open','b_open']].shift(1) -1
    dfsm = dfsm.dropna()

    covmat = np.cov(dfsm["s_returns"], dfsm["b_returns"])

# calculate measures now
    beta = covmat[0,1]/covmat[1,1]
    alpha = np.mean(dfsm["s_returns"])-beta*np.mean(dfsm["b_returns"])

# r_squared     = 1.0 - SS_res/SS_tot
    y = beta * dfsm["b_returns"] + alpha
    SS_res = np.sum(np.power(y - dfsm["s_returns"],2))
    SS_tot = covmat[0,0]*(len(dfsm) - 1) # SS_tot is sample_variance*(n-1)

    r_squared = 1.0 - SS_res/SS_tot
# Volatility for the full time and 1-year momentum
    volatility = np.sqrt(covmat[0,0])
    momentum = np.prod(1+dfsm["s_returns"].tail(12).values) - 1.0


    prd = 12.0 # used monthly returns; 12 periods to annualize
    alpha = alpha*prd
    volatility = volatility*np.sqrt(prd)

    print ("Beta, alpha, r_squared, volatility, momentum:")
    print (beta, alpha, r_squared, volatility, momentum)

    %pylab
    fig,ax = plt.subplots(1,figsize=(20,10))
    ax.scatter(dfsm["b_returns"], dfsm['s_returns'], label="Data points")
    beta,alpha = np.polyfit(dfsm["b_returns"], dfsm['s_returns'], deg=1)
    ax.plot(dfsm["b_returns"], beta*dfsm["b_returns"] + alpha, color='red', label="CAPM line")


    plt.title('Capital Asset Pricing Model, finding alphas and betas')
    plt.xlabel('Market return $R_m$', fontsize=14)
    plt.ylabel('Stock return $R_i$')
#     plt.text(0.05, 0.05, r'$R_i = \beta * R_m + \alpha$', fontsize=14)
    plt.legend()
    plt.grid(True)
    plt.show()
    return beta, y

In [13]:
beta, y = capm('2016-01-04', '2018-08-30','NVDA', '^GSPC')

Beta, alpha, r_squared, volatility, momentum:
2.166974886131238 0.61859069010838 0.1871663849759383 0.4010004992275091 0.6636636636636639
Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
